In [1]:
from toolkit.logger import Logger
from toolkit.classes import DataFrameCreator
from toolkit.pytorch_tools import CustomImageDataLoader
from toolkit.pytorch_tools import NeuralNetwork
from torchvision.transforms import Compose
from torchvision.transforms import Resize
from torchvision.transforms import CenterCrop
from torchvision.transforms import ToPILImage
from torchvision.transforms import ToTensor

import torch

import gc

In [2]:
torch.cuda.empty_cache()
gc.collect()
logger = Logger(__name__).get_logger()

In [3]:
data = DataFrameCreator()

In [4]:
data.load_dataset("./data/raw")
data.info()

[    INFO    ] DataFrame created from data/raw/Validation as validation data
[    INFO    ] DataFrame created from data/raw/Train as train data
[    INFO    ] DataFrame created from data/raw/Test as test data
[    INFO    ] Dataset loaded from data/raw folder
[    INFO    ] Name :          train
[    INFO    ] Type:           ('train',)
[    INFO    ] Columns:        ('path', 'filename', 'file', 'type', 'category', 'cat_idx')
[    INFO    ] Shape:          (300, 6)
[    INFO    ] Categories:     ('Non defective', 'Defective')
[    INFO    ] Path:           (PosixPath('data/raw/Train/Non defective'), PosixPath('data/raw/Train/Defective'))
[    INFO    ] File types:     ('jpg',)
[    INFO    ] Name :          validation
[    INFO    ] Type:           ('validation',)
[    INFO    ] Columns:        ('path', 'filename', 'file', 'type', 'category', 'cat_idx')
[    INFO    ] Shape:          (62, 6)
[    INFO    ] Categories:     ('Non defective', 'Defective')
[    INFO    ] Path:           (P

In [5]:
transform = Compose([
    ToPILImage(),
    Resize((227, 227)),
    CenterCrop((227, 227)),
    ToTensor()
])

loader = CustomImageDataLoader(dataset=data, image_col="file", label_col="cat_idx", transform=transform)
loader.create_dataloaders(batch_size=8)


[    INFO    ] CustomImageDataSet created
[    INFO    ] Dataloaders created


In [6]:
model = NeuralNetwork()
model.init_device()

[    INFO    ] NeuralNetwork(
  (layers): Sequential(
    (0): Conv2d(3, 96, kernel_size=(11, 11), stride=(4, 4))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(96, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(256, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU()
    (8): Conv2d(384, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU()
    (10): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU()
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (13): Flatten(start_dim=1, end_dim=-1)
    (14): Linear(in_features=9216, out_features=4096, bias=True)
    (15): ReLU()
    (16): Dropout(p=0.5, inplace=False)
    (17): Linear(in_features=4096, out_features=4096, bias=True)
    (18): ReLU

In [7]:
epochs = 2

model.train(epochs=epochs, train_loader=loader["train"], validation_loader=loader["validation"])

[    INFO    ] Model training started
[    INFO    ] Epoch:    0 Loss:  0.6951 Accuracy  0.4803  Validation loss:  0.6933   Validation accuracy:  0.4792
[    INFO    ] Epoch:    1 Loss:  0.6930 Accuracy  0.5033  Validation loss:  0.6930   Validation accuracy:  0.4479


In [8]:
model.results.data

,loss,accuracy,validation_loss,validation_acc
0,0.695133,0.480263,0.693343,0.479167
1,0.693038,0.503289,0.692979,0.447917
